In [1]:
filename = "GSE194227_mother.results.csv"
add_labels = 'mother_labels.csv'

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv(filename ,low_memory=False)

In [4]:
data.columns[data.isna().any()].tolist(), data.columns[data.isna().any()].tolist()

(['edgeR.logFC', 'edgeR.logCPM', 'edgeR.p.value', 'edgeR.adj.p.value'],
 ['edgeR.logFC', 'edgeR.logCPM', 'edgeR.p.value', 'edgeR.adj.p.value'])

In [5]:
data.drop(labels=['edgeR.logFC', 'edgeR.logCPM', 'edgeR.p.value', 'edgeR.adj.p.value', 'chr', 'start', 'stop'], axis=1, inplace=True)

In [6]:
data = data[(data.T != 0).any()]

In [7]:
data.shape

(2932684, 84)

In [8]:
names = data.columns

In [9]:
import re
new_names = []

for name in names:
    x = re.search(".*counts$", name)
    if x:
        new_names.append(name)

In [10]:
new_data = data[new_names]

In [11]:
labels = pd.read_csv(add_labels).transpose()[0]

In [12]:
transposed_data = new_data.transpose()

In [13]:
transposed_data.shape

(40, 2932684)

In [14]:
labels.shape

(40,)

In [15]:
labels.value_counts()

0    21
1    19
Name: 0, dtype: int64

In [16]:
transposed_data['label'] = labels

In [17]:
transposed_data = transposed_data.apply(pd.to_numeric)

In [18]:
X = transposed_data.drop(['label'], axis=1)
y = transposed_data['label']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

names = X.columns

data = scaler.fit_transform(X)

X = pd.DataFrame(data, columns=names)

In [19]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [20]:
fs = SelectKBest(score_func=mutual_info_classif, k=20)
X_selected = fs.fit_transform(X, y)

In [21]:
cols = fs.get_support(indices=True)

In [22]:
cols

array([  56694,  425451, 1042906, 1280477, 1333445, 1392019, 1473708,
       1570047, 1631434, 1854699, 1861625, 1959738, 1971224, 2044835,
       2258060, 2280858, 2408548, 2638177, 2739910, 2890407], dtype=int64)

In [23]:
X_selected = transposed_data.iloc[:,cols]

In [24]:
X_selected['label'] = labels

C:\Users\Deep Blade\AppData\Local\Temp\ipykernel_6812\949575838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_selected['label'] = labels


In [25]:
X_selected.sample(5)

,56992,445532,1063954,1302496,1355464,1414255,1496095,1609007,1670394,1894753,...,1999944,2011430,2101220,2348404,2371202,2507570,2737930,2858295,3009884,label
sPT436M.bam.counts,8,3,7,3,1,13,16,3,3,12,...,6,9,8,4,3,13,14,6,3,0
sPT409M.bam.counts,5,0,3,0,0,5,5,6,11,3,...,2,5,2,1,0,4,6,5,1,1
sPT404M.bam.counts,5,3,1,1,3,11,13,4,8,7,...,8,10,6,6,2,9,14,5,5,1
sPT427M.bam.counts,9,2,4,7,1,10,15,5,3,3,...,3,3,8,2,6,10,4,7,3,0
sPT416M.bam.counts,3,1,3,0,0,5,5,2,2,1,...,0,1,2,0,0,1,2,5,5,1


In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

names = X_selected.columns
d = scaler.fit_transform(X_selected)
scaled_df = pd.DataFrame(d, columns=names)

C:\Users\Deep Blade\anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Deep Blade\anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [27]:
scaled_df.to_csv("selected_M_best20.csv", encoding='utf-8', index=False)